In [ ]:
import pandas as pd
import xgboost as xgb

from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK
from hyperopt.pyll import scope as ho_scope
from hyperopt.pyll.stochastic import sample as ho_sample

from sklearn.model_selection import train_test_split 
 

In [ ]:
# pon los datos

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# espacio de hiperparametros

espacio ={'n_estimators': hp.quniform('n_estimators', 10, 1000, 25),
        'learning_rate': hp.uniform('learning_rate', 0.0001, 1.0),
        'max_depth': hp.quniform('x_max_depth', 4, 16, 1),
        'min_child_weight': hp.quniform ('x_min_child', 1, 10, 1),
        'subsample': hp.uniform ('x_subsample', 0.7, 1),
        'gamma' : hp.uniform ('x_gamma', 0.1,0.5),
        'reg_lambda' : hp.uniform ('x_reg_lambda', 0,1)
    }

In [ ]:
# funcion objetivo a minimizar

def objetivo(space):
    #print(space)
    clf = xgb.XGBRegressor(n_estimators =int(space['n_estimators']),
                           learning_rate = space['learning_rate'],
                           max_depth = int(space['max_depth']),
                           min_child_weight = space['min_child_weight'],
                           subsample = space['subsample'],
                           gamma = space['gamma'],
                           reg_lambda = space['reg_lambda'],
                           objective='reg:squarederror')

    eval_set=[(X_train, y_train), (X_test, y_test)]

    clf.fit(X_train, y_train,
            eval_set=eval_set, eval_metric="rmse", verbose=False)

    y_pred = clf.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred)**(0.5)
    #print ("SCORE:", np.sqrt(mse_scr))
    #change the metric if you like
    return {'loss':rmse, 'status': STATUS_OK }

In [ ]:
# minimiza

trials_reg = Trials()
best = fmin(fn=objetivo,
            space=space,
            algo=tpe.suggest,
            max_evals=10,
            trials=trials_reg)

In [ ]:
# mejores parametros

print (best)